In [1]:
import torch
from tools import config, Wrapper, DatasetQuora
from modelling.models import RNNCNNMatch
from modelling.templates import SimilarityTemplate

In [2]:
dataset = DatasetQuora(train_file=config.TRAIN_FILE,
                       test_file=config.TEST_FILE, 
                       sample_submission_file=config.SAMPLE_SUBMISSION_FILE)

In [3]:
rnn_cnn_match = SimilarityTemplate(query_model=RNNCNNMatch(), 
                                   verbose=config.VERBOSE, 
                                   loss_type='triplet')

In [4]:
optimizer = torch.optim.Adam(rnn_cnn_match.parameters())

In [ ]:
rcm = Wrapper(dataset=dataset, 
              model=rnn_cnn_match, 
              optimizer=optimizer, 
              model_name=config.MODEL_NAME, 
              generate_negatives_type='hard',
              batch_size=256)

In [ ]:
rcm.model

SimilarityTemplate(
  (query_embedding_layer): Embedding()
  (candidate_embedding_layer): Embedding()
  (query_model): RNNCNNMatch(
    (fully_connected): Linear(in_features=896, out_features=300, bias=True)
    (model): Sequential(
      (0): RNN(
        (rnn): LSTM(300, 256)
      )
      (1): CNN(
        (convolution_layer): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
      (2): CNN(
        (convolution_layer): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
      (3): CNN(
        (convolution_layer): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
        (pool_layer): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
      (4): CNN(
        (convolution_layer): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
        (pool_layer): MaxP

In [ ]:
%%time
rcm.collect_data(embeddings_weight_file=config.EMBEDDINGS_FILE)

/anaconda3/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  out = eval(code, glob, local_ns)
Collect embeddings:  37%|███▋      | 746948/1999996 [01:11<01:39, 12631.33it/s]

In [ ]:
rcm.train(verbose=config.VERBOSE)